In [0]:
!apt install cmake libopenmpi-dev zlib1g-dev 
#installing dependencies otherwise pip installation will throw error
!pip install stable-baselines
!apt install python-opengl

Reading package lists... Done
Building dependency tree       
Reading state information... Done
cmake is already the newest version (3.10.2-1ubuntu2).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2).
libopenmpi-dev is already the newest version (2.1.1-8).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-opengl is already the newest version (3.1.0+dfsg-1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


In [0]:
import os

import gym

from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines.deepq.policies import MlpPolicy
from stable_baselines import DQN
import numpy as np
import matplotlib.pyplot as plt
from stable_baselines.bench import Monitor
from stable_baselines.results_plotter import load_results, ts2xy


In [0]:
best_mean_reward, n_steps = -np.inf, 0

def callback(_locals, _globals):
    """
    Callback called at each step (for DQN an others) or after n steps (see ACER or PPO2)
    :param _locals: (dict)
    :param _globals: (dict)
    """
    global n_steps, best_mean_reward
    # Print stats every 1000 calls
    if (n_steps + 1) % 1000 == 0:
        # Evaluate policy performance
        x, y = ts2xy(load_results(log_dir), 'timesteps')
        if len(x) > 0:
            mean_reward = np.mean(y[-100:])
            print(x[-1], 'timesteps')
            print("Best mean reward: {:.2f} - Last mean reward per episode: {:.2f}".format(best_mean_reward, mean_reward))

            # New best model, you could save the agent here
            if mean_reward > best_mean_reward:
                best_mean_reward = mean_reward
                # Example for saving best model
                print("Saving new best model")
                _locals['self'].save(log_dir + 'best_model.pkl')
    n_steps += 1
    return False

In [0]:
# Create log dir
log_dir = "/tmp/gym/cartpole/"
os.makedirs(log_dir, exist_ok=True)


In [0]:
#Declaring  Environments

env = gym.make('CartPole-v1')
env = Monitor(env, log_dir, allow_early_resets=True)
#env = DummyVecEnv([lambda: env])

model = DQN(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=100000,callback=callback)
#model.save("deepq_cartpole")

# del model # remove to demonstrate saving and loading

# model = DQN.load("deepq_cartpole")

#obs = env.reset()
#while True:
#    action, _states = model.predict(obs)
#    obs, rewards, dones, info = env.step(action)
    #env.render()

/usr/local/lib/python3.6/dist-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


568 timesteps
Best mean reward: 91.79 - Last mean reward per episode: 21.04
1552 timesteps
Best mean reward: 91.79 - Last mean reward per episode: 21.56
--------------------------------------
| % time spent exploring  | 79       |
| episodes                | 100      |
| mean 100 episode reward | 21.5     |
| steps                   | 2132     |
--------------------------------------
2521 timesteps
Best mean reward: 91.79 - Last mean reward per episode: 21.53
3493 timesteps
Best mean reward: 91.79 - Last mean reward per episode: 26.27
4567 timesteps
Best mean reward: 91.79 - Last mean reward per episode: 32.25
5503 timesteps
Best mean reward: 91.79 - Last mean reward per episode: 39.22
6372 timesteps
Best mean reward: 91.79 - Last mean reward per episode: 46.61
7385 timesteps
Best mean reward: 91.79 - Last mean reward per episode: 55.26
8439 timesteps
Best mean reward: 91.79 - Last mean reward per episode: 64.00
--------------------------------------
| % time spent exploring  | 11     

In [0]:
def movingAverage(values, window):
    """
    Smooth values by doing a moving average
    :param values: (numpy array)
    :param window: (int)
    :return: (numpy array)
    """
    weights = np.repeat(1.0, window) / window
    return np.convolve(values, weights, 'valid')


def plot_results(log_folder, title='Learning Curve For CartPole'):
    """
    plot the results

    :param log_folder: (str) the save location of the results to plot
    :param title: (str) the title of the task to plot
    """
    x, y = ts2xy(load_results(log_folder), 'timesteps')
    y = movingAverage(y, window=50)
    #Truncate x
    x = x[len(x) - len(y):]

    fig = plt.figure(title)
    plt.plot(x, y)
    plt.xlabel('Number of Timesteps')
    plt.ylabel('Rewards')
    plt.title(title + " Smoothed")
    plt.title(title)
    plt.show()


In [0]:
plot_results(log_dir)